# Определение наиболее выгодного региона нефтедобычи

## Содержание
1. [Описание проекта](#id_description)
2. [Загрузка и подготовка данных](#id_preprocessing)
3. [Обучение и проверка модели](#id_learning)
4. [Подготовка к расчёту прибыли](#id_benifit_calc)
5. [Расчёт прибыли и рисков](#id_risk_benifit)
6. [Общий вывод](#id_conclusion)

## Описание проекта
<a id="id_description"></a>

**Описание проекта**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

Данные геологоразведки трёх регионов находятся в файлах:

   - /datasets/geo_data_0.csv
   - /datasets/geo_data_1.csv
   - /datasets/geo_data_2.csv
   - id — уникальный идентификатор скважины;
   - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
   - product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

   - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
   - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
   - Бюджет на разработку скважин в регионе — 10 млрд рублей.
   - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
   - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных
<a id="id_preprocessing"></a>

In [2]:
# загрузка данных
try:
    geo_0 = pd.read_csv('data-files/geo_data_0.csv')  # Локальный путь
    geo_1 = pd.read_csv('data-files/geo_data_1.csv')  # Локальный путь
    geo_2 = pd.read_csv('data-files/geo_data_2.csv')  # Локальный путь
#    df_churn = pd.read_csv('data-files/Churn.csv',index_col='RowNumber')  # Локальный путь
except:
    geo_0 = pd.read_csv('/datasets/geo_data_0.csv') # Серверный путь
    geo_1 = pd.read_csv('/datasets/geo_data_1.csv') # Серверный путь
    geo_2 = pd.read_csv('/datasets/geo_data_2.csv') # Серверный путь
#    df_churn = pd.read_csv('/datasets/Churn.csv',index_col='RowNumber')  # Серверный путь

Посмотрим данные

In [3]:
geo_0.info()
print()
print('Количество явных дубликатов:',geo_0.duplicated().sum())
geo_0.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов: 0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
geo_1.info()
print()
print('Количество явных дубликатов:',geo_1.duplicated().sum())
geo_1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов: 0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [5]:
geo_2.info()
print()
print('Количество явных дубликатов:',geo_2.duplicated().sum())
geo_2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Количество явных дубликатов: 0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Разделим данные на валидационную и обучающую выборки:

In [6]:
valid_size = 0.25
features_name = ['f0','f1','f2']
target_name = 'product'

In [7]:
def split_stand(data,features_name,target_name,valid_size):
    features = data[features_name]
    target = data[target_name]
    
    # Разбивка на обучающую и валидационную выборки
    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                                  target,
                                                                                  test_size=valid_size,
                                                                                  random_state=112345
                                                                                 )
    # стандартизация
    scaler = StandardScaler()
    scaler.fit(features_train) 
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
        
    # Проверка на соответсвие размеров выборок
    print()
    print('Размер валидационной выборки:')
    print('features:',features_valid.shape)
    print('target:',target_valid.shape)

    print('Размер обучающей выборки:')
    print('features:',features_train.shape)
    print('target:',target_train.shape)
    
    return features_train, features_valid, target_train, target_valid

In [8]:
print('Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_0')
features_train_0, features_valid_0, target_train_0, target_valid_0 = split_stand(geo_0, 
                                                                                 features_name, 
                                                                                 target_name, 
                                                                                 valid_size)
print()
print('Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_1')
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_stand(geo_1, 
                                                                                 features_name, 
                                                                                 target_name, 
                                                                                 valid_size)
print()
print('Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_2')
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_stand(geo_2, 
                                                                                 features_name, 
                                                                                 target_name, 
                                                                                 valid_size)

Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_0

Размер валидационной выборки:
features: (25000, 3)
target: (25000,)
Размер обучающей выборки:
features: (75000, 3)
target: (75000,)

Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_1

Размер валидационной выборки:
features: (25000, 3)
target: (25000,)
Размер обучающей выборки:
features: (75000, 3)
target: (75000,)

Разбивка на обучающую и валидационную выборки + стандартизация для данных geo_2

Размер валидационной выборки:
features: (25000, 3)
target: (25000,)
Размер обучающей выборки:
features: (75000, 3)
target: (75000,)


**Вывод**

Имеются три набора данных о месторождениях нефти в трёх регионах: в каждом наборе 10 000 месторождений и для каждого месторождения в данных есть уникальный идентификатор (id), три параметра качества нефти (f0, f1, f2) и объём её запасов (product).

Данные чистые, без аномалий и дубликатов и не требуют предобработки.

Для дальнейшего исследования данные были стандартизированы и поделены на обучающую и валидационную выборки (в соотношении 3:1).



## Обучение и проверка модели
<a id="id_learning"></a>

In [9]:
def prediction(features_train,target_train,features_valid,target_valid):
    model = LinearRegression()

    model.fit(features_train,target_train)
    prediction = model.predict(features_valid)

    rmse = mean_squared_error(target_valid,prediction,squared=False)

    return prediction,rmse

In [10]:
print('Обучение и предсказания линейной регрессии для данных geo_0')
prediction_valid_0,rmse_0 = prediction(features_train_0,
                                 target_train_0,
                                 features_valid_0,
                                 target_valid_0)
print("Средний запас сырья в скважине для валидационной выборки данных geo_0: {:,.2f}".format(prediction_valid_0.mean()))
print("RMSE для валидационной выборки данных geo_0: {:,.2f}".format(rmse_0))

print()
print('Обучение и предсказания линейной регрессии для данных geo_1')
prediction_valid_1,rmse_1 = prediction(features_train_1,
                                       target_train_1,
                                       features_valid_1,
                                       target_valid_1)
print("Средний запас сырья в скважине для валидационной выборки данных geo_1: {:,.2f}".format(prediction_valid_1.mean()))
print("RMSE для валидационной выборки данных geo_1: {:,.2f}".format(rmse_1))

print()
print('Обучение и предсказания линейной регрессии для данных geo_2')
prediction_valid_2,rmse_2 = prediction(features_train_2,
                                       target_train_2,
                                       features_valid_2,
                                       target_valid_2)
print("Средний запас сырья в скважине для валидационной выборки данных geo_2: {:,.2f}".format(prediction_valid_2.mean()))
print("RMSE для валидационной выборки данных geo_2: {:,.2f}".format(rmse_2))

Обучение и предсказания линейной регрессии для данных geo_0
Средний запас сырья в скважине для валидационной выборки данных geo_0: 92.20
RMSE для валидационной выборки данных geo_0: 37.89

Обучение и предсказания линейной регрессии для данных geo_1
Средний запас сырья в скважине для валидационной выборки данных geo_1: 69.02
RMSE для валидационной выборки данных geo_1: 0.88

Обучение и предсказания линейной регрессии для данных geo_2
Средний запас сырья в скважине для валидационной выборки данных geo_2: 95.00
RMSE для валидационной выборки данных geo_2: 39.91


**Вывод**

Обучена модель линейной регресии на данных для каждого региона, далее с помощью этой модели следаны предсказания для валидационной выборки. На валидационной выборке наименьший уровень ошибки у региона из файла geo_1. Однако, это же регион имеет и наименьшие средние запасы нефти. Наибольшие среднии запасы показывает регион geo_2.

## Подготовка к расчёту прибыли
<a id="id_benifit_calc"></a>

In [11]:
# запишем условия в переменных
OILWELLS_EXPLORE = 500
OILWELLS_EXPLOIT = 200
TOTAL_BUDGET = 10_000_000_000
PROFIT_PER_KILOBARREL = 450_000
RISK_LEVEL = 0.025
ONE_OILWELL_BUDGET = TOTAL_BUDGET / OILWELLS_EXPLOIT
LOSS_FREE_BARRELS = ONE_OILWELL_BUDGET // PROFIT_PER_KILOBARREL

In [13]:
print("Бюджет для разработки одной скважины: {:,.2f}".format(ONE_OILWELL_BUDGET))

Бюджет для разработки одной скважины: 50,000,000.00


In [14]:
print("Объём сырья для безубыточной разработки новой скважины (тыс.баррель): {:,.2f}".format(LOSS_FREE_BARRELS))

Объём сырья для безубыточной разработки новой скважины (тыс.баррель): 111.00


In [15]:
print("Средний запас сырья в скважине в регионе geo_0 (тыс.баррель): {:,.2f}".format(prediction_valid_0.mean()))
print("Средний запас сырья в скважине в регионе geo_1 (тыс.баррель): {:,.2f}".format(prediction_valid_1.mean()))
print("Средний запас сырья в скважине в регионе geo_2 (тыс.баррель): {:,.2f}".format(prediction_valid_2.mean()))

Средний запас сырья в скважине в регионе geo_0 (тыс.баррель): 92.20
Средний запас сырья в скважине в регионе geo_1 (тыс.баррель): 69.02
Средний запас сырья в скважине в регионе geo_2 (тыс.баррель): 95.00


**Вывод**

Чтобы бурение скважины было безубыточным, запас нефти в месторожении должен быть не менее 111 тыс. баррелей. 
Однако, средние значения по объемам запасов ниже этого уровня для всех регионов. Осбено низкий средний уровень запасов в регионе geo_1.

## Расчёт прибыли и рисков 
<a id="id_risk_benifit"></a>

In [96]:
# функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:
def profit_oilwell(prediction,target):
    try:
        top_best = pd.concat([pd.Series(prediction),
                              target.reset_index(drop=True)],
                             axis=1,
                             ignore_index=True)
    except:
        top_best = pd.concat([prediction,target],
                             axis=1,
                             ignore_index=True)
                             
    top_best = (top_best.sort_values(by=[0],ascending=False)
                [:OILWELLS_EXPLOIT]
               )
    
    total_profit = (top_best[1].sum() *  PROFIT_PER_KILOBARREL) - (ONE_OILWELL_BUDGET * len(top_best[1]))
    return total_profit

In [97]:
print("Прибыль 200 месторождений в регионе geo_0: {:,.2f}".format(profit_oilwell(prediction_valid_0,
                                                                                 target_valid_0)))

print("Прибыль 200 месторождений в регионе geo_1: {:,.2f}".format(profit_oilwell(prediction_valid_1,
                                                                                 target_valid_1)))

print("Прибыль 200 месторождений в регионе geo_2: {:,.2f}".format(profit_oilwell(prediction_valid_2,
                                                                                 target_valid_2)))

Прибыль 200 месторождений в регионе geo_0: 3,098,311,591.85
Прибыль 200 месторождений в регионе geo_1: 2,415,086,696.68
Прибыль 200 месторождений в регионе geo_2: 2,022,769,164.46


In [108]:
# функция для расчёта рисков убытков и прибыли с применением bootstrap:
def bootstrap_func(prediction,target):
    data = pd.concat([pd.Series(prediction),
                      target.reset_index(drop=True)],
                     axis=1,
                     ignore_index=True)
    state = np.random.RandomState(12345)
    
    values = []
    for i in range(1000):
        subsample = data.sample(n=OILWELLS_EXPLORE, replace=True, random_state=state)
        values.append(profit_oilwell(subsample[0],subsample[1]))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
   # risk = len(values[values < 0]) / len(values)
    risk = (values < 0).mean()
    
    print("Средняя прибыль: {:,.2f} руб.".format(mean))
    print("Нижняя граница доверительного интервала: {:,.2f} руб.".format(lower))
    print("Верхняя граница доверительного интервала: {:,.2f} руб.".format(upper))
    print("Риск убытков: {:,.2%}.".format(risk))

In [109]:
print('Регион geo_0')
bootstrap_func(prediction_valid_0,target_valid_0)

print()
print('Регион geo_1')
bootstrap_func(prediction_valid_1,target_valid_1)

print()
print('Регион geo_2')
bootstrap_func(prediction_valid_2,target_valid_2)

Регион geo_0
Средняя прибыль: 398,630,223.38 руб.
Нижняя граница доверительного интервала: -117,131,758.06 руб.
Верхняя граница доверительного интервала: 946,493,958.44 руб.
Риск убытков: 6.60%.

Регион geo_1
Средняя прибыль: 426,613,824.14 руб.
Нижняя граница доверительного интервала: 35,168,690.78 руб.
Верхняя граница доверительного интервала: 865,813,252.59 руб.
Риск убытков: 1.90%.

Регион geo_2
Средняя прибыль: 277,655,644.26 руб.
Нижняя граница доверительного интервала: -215,551,294.74 руб.
Верхняя граница доверительного интервала: 822,779,815.26 руб.
Риск убытков: 11.90%.


## Общий вывод
<a id="id_conclusion"></a>

Уровень риска ниже установленого значения только для региона geo_2.

Также для этого регион наибольшая средняя выручка, а 95% доверительные интервал не уходит в отрицательную зону.

Соответсвенно, для разработки месторождений предпочтительней регион geo_1.

Средняя прибыль в регионе geo_1:  426,613,824.14 руб.

Риск убытков в регионе geo_1: 1.90%.

In [114]:
geo_1['product'].median()

57.08562464628662